#Cél
A modellt finomhangoljuk arra a feladatra, hogy 2 mondatról eldöntse, a jelentésük ugyanaz-e, vagy sem.

# Fontos csomagok letöltése

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

# Tanítóhalmaz beszerzése
Itt a HUB-ról letöltött Microsoft Research Paraphrase Corpus adathalmazt használjuk, amely a GLUE benchmark egyik adathalmaza.


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

# Tokenizer
Most, hogy megszereztük a tanítóhalmazt, azt is látjuk, hogy ez egy dictionary, melynek 3 eleme van: train, validation, test.
Az AutoTokenizer beszerzi ebből az adathalmazból a tokenizált adatot.
Kiegészíti tehát az adathalmazokat a következő feature-ökkel: input_ids (ami megadja a tokenek id-ját), token_type_ids (ami megadja az id-k típusát), attention_mask (ami megmondja, mikre kell figyelni). \
Még nem csináltunk paddinget, mert azt jobb akkor, amikor már batchekre van bontva. \
A .map() függvény egy transzformációs függvényt alkalmaz minden példára a datasetben, és visszaad egy új, átalakított datasetet. \
A batched=True azt csinálja, hogy a .map() egyszerre több példát (egy batch-et) adjon át a függvénynek.


In [ ]:
from transformers import AutoTokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

#Dynamic Padding
A Paddinget külön kezeljük, mert:
Érdemes batchenként meghatározni, hogy melyik a leghosszabb mondat, és ahhoz igazítani a többi mondat tokenhosszát.
Ehhez előbb létre kell hozni a batcheket, amit a collate() függvénnyel lehet létrehozni, és kell mellé a padding is, így a DataCollatorWithPadding metódust fogjuk hozzá alkalmazni. \

A létrehozott data_collator képes arra, hogy a létrehozott batchekben paddingeljen, tehát automatikusan megállapítsa a legtöbb tokennel rendelkező mondatot.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


#A training hyperparaméterei
Egyedül azt kell beállítanunk, hogy hova mentődjön a tanított modell a checkpointjaival. A többi maradhat alapértelmezetten.

A TrainingArguments egy konfigurációs objektum a Hugging Facen, ami megmondja a Trainernek, hogy hogyan tanítsa a modellt:
*   Hova mentse a modelleket és logokat
*   Hány epochot futtasson
*   Mekkora legyen a batch size
*   Milyen gyakran mentsen vagy értékeljen, stb.

A "test-trainer" paraméter azt jelenti, hogy minden mentést, logot, checkpointot a test-trainer/ nevű mappába fig elmenteni. \
A report_to[] paraméter azt szabályozza, hovát küldje a Trainer a tanítási naplókat, statisztikákat. Ezzel a beállítással most nem küld semmilyen logot semmilyen külső szolgáltatásnak.




In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", report_to=[])

#A model definiálása
Most a AutoModelForSequenceClassification osztály használjuk, legyen a num_labels=2

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

#Az evaluate definiálása
Szükségünk van arra, hogy a modell kiértékelje a tanítást bizonyos időközénként. Ehhez a comput_metrics() functiont hívjuk segítségül.

In [ ]:
import evaluate
import numpy as np
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#Trainer definiálása, ami elvégzi a tanítást
Most, hogy minden szükséges paramétert definiáltunk, beadhatjuk a Trainernek, aki elvégzi a tanítást a modellen.
A végén meghívjuk, hogy végezze el a tanítást.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

trainer.train()